In [2]:
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import xgboost

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [4]:
try:
    train = pd.read_csv("./train_for_crossval.csv", delimiter=",", sep='.')
except:
    train = pd.read_csv("./sample_data/train_for_crossval.csv", delimiter=",", sep='.')

In [5]:
train

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,anomaly_col
0,B,A,A,C,B,B,A,E,E,I,...,0.449762,0.281378,0.566167,0.564149,0.094746,0.347672,0.948445,0.410370,7.200575,Norm
1,A,A,A,C,B,D,A,E,A,I,...,0.562349,0.349834,0.953103,0.808620,0.663709,0.503259,0.692824,0.760255,9.857370,Norm
2,A,B,A,C,B,B,A,E,E,G,...,0.339495,0.365626,0.471769,0.318679,0.249393,0.352284,0.334168,0.265968,8.476475,Norm
3,A,B,A,C,B,B,A,E,C,I,...,0.607722,0.653107,0.555535,0.671799,0.573051,0.391958,-0.072907,0.524188,8.015375,Norm
4,B,A,A,C,B,D,A,E,C,H,...,0.902596,0.892918,0.721439,0.346334,0.119194,0.413215,0.423083,0.264100,10.375199,Norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,A,A,A,C,B,D,A,D,A,F,...,0.594021,1.032560,0.905694,0.852805,0.809571,0.358780,0.823077,0.865032,8.521849,Anomaly
89996,B,A,A,D,B,B,A,E,E,B,...,0.361353,0.900748,0.468781,0.267659,0.695161,0.421088,0.841941,0.615257,8.988227,Anomaly
89997,B,A,B,C,B,D,A,E,A,F,...,0.705946,0.896323,0.639372,0.598032,0.490381,0.134411,0.764455,0.240164,9.021154,Norm
89998,B,A,A,C,B,D,A,E,G,J,...,0.331300,0.369129,0.601829,0.506245,0.583071,0.429652,0.776692,0.662461,7.701070,Norm


In [6]:
train, t = train_test_split(train, test_size=0.9)

In [7]:
X = train.drop(["target"], axis=1)
y = train["target"] * 100

In [8]:
num_train = train.select_dtypes([int, float])
cat_train = train.select_dtypes(object)

num = list(num_train.drop(['target'],axis=1))
cat = list(cat_train)

In [9]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OrdinalEncoder

rmse = make_scorer(mean_squared_error, squared=False)

pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler()),  
    ('normal', PowerTransformer()), 
])
pipeline_cat = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoding', OrdinalEncoder()),
    ('encoding', OneHotEncoder(handle_unknown='ignore')),
])
preprocessor = ColumnTransformer(n_jobs=-1,
    transformers=[
        ('num', pipeline_num, num),
        ('cat', pipeline_cat, cat),
        ], remainder="passthrough")

transformed = preprocessor.fit_transform(X)

# svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
# transformed = svd.fit_transform(transformed)

def objective(trial):
    
    param_model = {
        # 'tree_method':'gpu_hist',
        'eval_metric':trial.suggest_categorical("eval_metric", ['rmsle', 'rmse']), #rmse
        'n_estimators': 500,
        'random_state': trial.suggest_categorical("random_state", [0, 42]), 
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1),
        'max_depth': trial.suggest_int('max_depth', 4, 6),
        'objective': trial.suggest_categorical("objective", ['reg:squarederror']),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 100),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'subsample': trial.suggest_float('subsample', 0.5, 0.9)
    }
    
    pipeline_ridge = Pipeline(steps=[
                                    #  ('preprocessor', preprocessor), 
#                                      ('svd', TruncatedSVD(random_state=42, **param_svd)), 
                                     ('model', xgboost.XGBRegressor(**param_model)),
                                     ])

    rmse_mean_cv = cross_val_score(pipeline_ridge, transformed, y, cv=5, scoring=rmse, n_jobs=-1)
    print(rmse_mean_cv)

    return rmse_mean_cv.mean()

In [10]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=10)

[I 2021-08-30 11:10:32,591] A new study created in memory with name: no-name-a7b8a104-8787-4c8b-a5d3-a095f72a78f7
[I 2021-08-30 11:11:25,409] Trial 0 finished with value: 78.80729709123753 and parameters: {'eval_metric': 'rmsle', 'random_state': 0, 'colsample_bytree': 0.8710667739512141, 'learning_rate': 0.2372664547361594, 'max_depth': 4, 'objective': 'reg:squarederror', 'reg_alpha': 5.2386603748866225, 'reg_lambda': 98, 'subsample': 0.785390046752009}. Best is trial 0 with value: 78.80729709123753.


[78.35033273 80.26859479 77.33537765 78.04913777 80.03304251]


[I 2021-08-30 11:12:08,159] Trial 1 finished with value: 75.42417597675299 and parameters: {'eval_metric': 'rmse', 'random_state': 0, 'colsample_bytree': 0.6706685351406318, 'learning_rate': 0.06636975003160032, 'max_depth': 4, 'objective': 'reg:squarederror', 'reg_alpha': 23.703516737785538, 'reg_lambda': 83, 'subsample': 0.5745730910998585}. Best is trial 1 with value: 75.42417597675299.


[74.80098795 76.50665198 73.59164797 75.48889512 76.73269686]


[I 2021-08-30 11:13:03,610] Trial 2 finished with value: 88.9428745734966 and parameters: {'eval_metric': 'rmsle', 'random_state': 42, 'colsample_bytree': 0.8145956844711555, 'learning_rate': 0.7453995941295726, 'max_depth': 5, 'objective': 'reg:squarederror', 'reg_alpha': 45.220360727965925, 'reg_lambda': 57, 'subsample': 0.7834974295834356}. Best is trial 1 with value: 75.42417597675299.


...

[94.13604536 95.07017419 92.23244185 93.48134813 97.66571937]


[I 2021-08-30 11:17:39,466] Trial 8 finished with value: 79.09230239125259 and parameters: {'eval_metric': 'rmsle', 'random_state': 42, 'colsample_bytree': 0.6591737313139487, 'learning_rate': 0.2500238296307967, 'max_depth': 4, 'objective': 'reg:squarederror', 'reg_alpha': 99.6990057463686, 'reg_lambda': 90, 'subsample': 0.6109129285813852}. Best is trial 1 with value: 75.42417597675299.


[78.9244834  79.89640834 77.60366986 79.01229565 80.0246547 ]


[I 2021-08-30 11:18:24,697] Trial 9 finished with value: 78.93236261338599 and parameters: {'eval_metric': 'rmsle', 'random_state': 42, 'colsample_bytree': 0.4800505601585979, 'learning_rate': 0.15666160003620883, 'max_depth': 5, 'objective': 'reg:squarederror', 'reg_alpha': 95.92063279303679, 'reg_lambda': 22, 'subsample': 0.5470726693802097}. Best is trial 1 with value: 75.42417597675299.


[78.10697021 79.33551831 77.50870008 78.30021151 81.41041296]


In [26]:
best_params = study.best_trial.params
best_params

{'eval_metric': 'rmse',
 'random_state': 0,
 'colsample_bytree': 0.6706685351406318,
 'learning_rate': 0.06636975003160032,
 'max_depth': 4,
 'objective': 'reg:squarederror',
 'reg_alpha': 23.703516737785538,
 'reg_lambda': 83,
 'subsample': 0.5745730910998585}

# save configs

In [ ]:
import yaml
import os


directory = './Configs'
if not os.path.exists(directory):
    os.makedirs(directory)
    
    
with open('./Configs/params_model_xgb.yml', 'w') as outfile:
    yaml.dump(best_params, outfile, default_flow_style=False)